In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class DeepFM(nn.Module):
    def __init__(self, field_dims, embed_dim, hidden_dims):
        super(DeepFM, self).__init__()
        self.num_fields = len(field_dims)
        self.embeddings = nn.ModuleList([nn.Embedding(field_dim, embed_dim) for field_dim in field_dims])

        # FM 部分的一阶部分
        self.linear = nn.Embedding(sum(field_dims), 1)
        
        # DNN 部分
        input_dim = self.num_fields * embed_dim
        all_dims = [input_dim] + hidden_dims
        dnn_layers = []
        for i in range(len(all_dims) - 1):
            dnn_layers.append(nn.Linear(all_dims[i], all_dims[i + 1]))
            dnn_layers.append(nn.ReLU())
        self.mlp = nn.Sequential(*dnn_layers)
        self.fc = nn.Linear(all_dims[-1], 1)

    def forward(self, x):
        x = x.long()
        # FM 一阶部分
        linear_term = self.linear(x).sum(dim=1)

        # 嵌入向量
        embeds = [self.embeddings[i](x[:, i]) for i in range(self.num_fields)]
        embeds = torch.stack(embeds, dim=1)

        # FM 二阶交互部分
        fm_sum_square = torch.sum(embeds, dim=1) ** 2
        fm_square_sum = torch.sum(embeds ** 2, dim=1)
        fm_second_order = 0.5 * torch.sum(fm_sum_square - fm_square_sum, dim=1, keepdim=True)

        # DNN 部分
        dnn_input = embeds.view(embeds.size(0), -1)  # 将嵌入平坦化
        dnn_output = self.fc(self.mlp(dnn_input))

        # 最终预测
        y = torch.sigmoid(linear_term + fm_second_order + dnn_output)
        return y

# 示例参数
field_dims = [10, 10, 10]  # 每个特征的不同取值个数
model = DeepFM(field_dims, embed_dim=4, hidden_dims=[10, 10])

# 假数据
x = torch.randint(0, 10, (4, 3))  # batch_size=4, num_features_per_sample=3
y = model(x)
print(y)

tensor([[0.9826],
        [0.9417],
        [0.8813],
        [0.9467]], grad_fn=<SigmoidBackward0>)
